# SQLite
Refs:
  * [Python documentation](https://docs.python.org/3/library/sqlite3.html)
  * [Main documentation](https://www.sqlite.org/index.html)

## Types  
SQLite is type less, the basic types that it recognizes are -
  * TEXT
  * NUMERIC
  * INTEGER
  * REAL
  * BLOB (aka NONE)

However when creating tables it is still a good idea to specify the data types to serve as documentation. SQLite does some fancy parsing of the type specified by the user and then assigns its own type from the above primitives. This is called [column affinity](https://www.sqlite.org/datatype3.html#affname).

Date types are usually stored as TEXT. In order to use the date/time functions this needs to be in ISO 8601 format.

Booleans are usually stored as INTEGERs, with 0 to represent False and 1 to represent True.

TODO: Concept of primary keys.

## Queries
The typical flow is to first get a `Connection` object, and then get a `Cursor` from the connection, and then call `execute` on the cursor. However, this pattern is so common, that it is possible to call `execute` directly on the `Connection` object. This will return a "loaded" `Cursor` object.

In [ ]:
import sqlite3
from datetime import datetime

In [2]:
# conn = sqlite3.connect("~/temp/learn.db")
conn = sqlite3.connect(":memory:")

In [3]:
conn.execute(
"""
CREATE TABLE IF NOT EXISTS video_games (
  id CHAR(16) PRIMARY KEY,
  title TEXT,
  released_on TEXT,
  rating INTEGER,
  price REAL,
  is_on_pc BOOL
)
"""
)

In [4]:
data = [
    (
        "4wuXytmedmydhomj", 
        "Stray", 
        datetime(2023, 8, 10).isoformat(timespec="seconds"),
        5,
        23.99,
        1
    ),
    (
        "Q25wQumhCed7Eobs",
        "Remnant II - Standard Edition",
        datetime(2023, 7, 25).isoformat(timespec="seconds"),
        4,
        49.99,
        0
    )
]

In [5]:
conn.executemany("INSERT INTO video_games VALUES (?, ?, ?, ?, ?, ?)", data)
conn.commit()

In [6]:
for row in conn.execute("SELECT * FROM video_games"):
    print(row)

('4wuXytmedmydhomj', 'Stray', '2023-08-10T00:00:00', 5, 23.99, 1)
('Q25wQumhCed7Eobs', 'Remnant II - Standard Edition', '2023-07-25T00:00:00', 4, 49.99, 0)


In addition to the ? placeholder, SQLite also supports named placeholders.

In [7]:
params = dict(rating=4, is_on_pc=True)
for row in conn.execute("SELECT * FROM video_games WHERE rating >= :rating AND is_on_pc = :is_on_pc", params):
    print(row)

('4wuXytmedmydhomj', 'Stray', '2023-08-10T00:00:00', 5, 23.99, 1)


By default SQLite returns rows as a tuple with fields in the same position as they were created in. This can be made more convenient using row factories. There is a builtin row factory called `Row` that provides indexed and case-insensitive named access to columns. If I don't like this I can create any row factory that I want.

In [8]:
conn.row_factory = sqlite3.Row

In [16]:
cur = conn.execute("SELECT * FROM video_games")
row = cur.fetchone()
row.keys()

['id', 'title', 'released_on', 'rating', 'price', 'is_on_pc']

In [17]:
print(row["id"], row[0])
print(row["title"], row[1])

4wuXytmedmydhomj 4wuXytmedmydhomj
Stray Stray


In [18]:
type(cur)

sqlite3.Cursor

In [19]:
cur.description

(('id', None, None, None, None, None, None),
 ('title', None, None, None, None, None, None),
 ('released_on', None, None, None, None, None, None),
 ('rating', None, None, None, None, None, None),
 ('price', None, None, None, None, None, None),
 ('is_on_pc', None, None, None, None, None, None))

In [13]:
from dataclasses import dataclass

In [14]:
@dataclass
class VideoGame:
    id: str
    title: str
    released_on: datetime
    rating: int
    price: float
    is_on_pc: bool

In [24]:
def dataclass_factory(cursor, row):
    colnames = [col[0] for col in cur.description]
    return VideoGame(
        id=row[colnames.index("id")],
        title=row[colnames.index("title")],
        released_on=datetime.fromisoformat(row[colnames.index("released_on")]),
        rating=row[colnames.index("rating")],
        price=row[colnames.index("price")],
        is_on_pc=row[colnames.index("is_on_pc")]
    )

In [25]:
conn.row_factory = dataclass_factory

In [26]:
cur = conn.execute("SELECT * FROM video_games")
row = cur.fetchone()
row

VideoGame(id='4wuXytmedmydhomj', title='Stray', released_on=datetime.datetime(2023, 8, 10, 0, 0), rating=5, price=23.99, is_on_pc=1)

In [27]:
conn.close()

In [28]:
cur.fetchone()

ProgrammingError: Cannot operate on a closed database.